### todo
1. all sections

In [ ]:


# special eval
# train overlap performance
# load_and_run('baseline_naive_e12_val')

_, _, (train, _, _) = load_guardian_splits(k_json_folder)

In [ ]:
def make_set_inclusion_filter_fcn(train_set: List[BaseClue]):
    s = set()
    for c in train_set:
        s.add(c.soln_with_spaces)

    # return False to omit
    def filter_fcn(mp: vt.ModelPrediction):
        if mp.target in s:
            return False
        return True
    return filter_fcn

In [ ]:
# subset that does not overlap with train (exact match)
load_and_run('baseline_naive_e12_val',
             label='baseline_naive_e12_val_diff_exact',
             filter_fcn=make_set_inclusion_filter_fcn(train))
load_and_run('baseline_naive_e12_test',
             label='baseline_naive_e12_test_diff_exact',
             filter_fcn=make_set_inclusion_filter_fcn(train))

In [ ]:
###
# prepare for set that does not overlap, fuzzily
##
# this result not in paper
def make_set_inclusion_filter_fcn_fuzz(train_set: List[BaseClue]):
    s = set()
    for c in train_set:
        soln = c.soln_with_spaces
        if soln.endswith('es'):
            s.add(soln[:-2])
        if soln.endswith('s'):
            s.add(soln[:-1])
        s.add(soln + 'es')
        s.add(soln + 's')
        s.add(soln)

    # return False to omit
    def filter_fcn(mp: vt.ModelPrediction):
        if mp.target in s or mp.target[:-1] in s or mp.target[:-2] in s:
            return False
        return True
    return filter_fcn

In [ ]:
load_and_run('baseline_naive_e12_val',
             label='baseline_naive_e12_val_diff_plurals',
             filter_fcn=make_set_inclusion_filter_fcn_fuzz(train))
load_and_run('baseline_naive_e12_test',
             label='baseline_naive_e12_test_diff_plurals',
             filter_fcn=make_set_inclusion_filter_fcn_fuzz(train))

In [ ]:
###
# disjoint
###
# not reported in paper
_, _, (train_disj, val_disj, _) = load_guardian_splits_disjoint_hash(k_json_folder)

load_and_run('baseline_disj1_e6_val',
             label='baseline_disj1_e6_val_diff',
             filter_fcn=make_set_inclusion_filter_fcn_fuzz(train_disj))


